In [11]:
!conda install -y spacy
!conda install -y bs4
!pip install /home/ec2-user/SageMaker/packages/blingfire-0.1.8.tar.gz
!conda install -y pandas

Solving environment: \ ^C
failed with initial frozen solve. Retrying with flexible solve.

CondaError: KeyboardInterrupt

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0



# All requested packages already installed.

Processing /home/ec2-user/SageMaker/packages/blingfire-0.1.8.tar.gz
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/packages/blingfire-0.1.8.tar.gz'



In [1]:
import os
# import spacy
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import re
# from sentencex import segment
from blingfire import text_to_sentences
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import pandas as pd

In [ ]:
# nlp = spacy.load('en_core_web_sm')
# nlp.add_pipe('sentencizer')
# nlp.select_pipes(enable=["sentencizer"])

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [6]:
def check_inflation(line):
    pattern = r'\binflation\b'
    matches = re.search(pattern, line, re.IGNORECASE)
    if matches:
        return True
    else:
        return False

In [21]:
base_data_dir = "data/Historical_News_test_atlanta"
out_dir = "data"  # Specify your output directory


def process_file(file):
    
    none_text = 0
    total_sents = 0
    inflation_sents = 0
    
    tree = ET.parse(os.path.join(base_data_dir, file))
    root = tree.getroot()
    
    hidden_text_element = root.findall(".//*[@HTMLContent='true']")
    if hidden_text_element is None or hidden_text_element == []:
        none_text += 1
        return  none_text, total_sents, [], None, None
    
    hidden_text_html = ""
    for el in hidden_text_element:
        hidden_text_html += ("\n" + el.text)
    
    soup = BeautifulSoup(hidden_text_html, 'html.parser')
    text = soup.get_text(separator='\n').strip()
    
    # Replace multiple consecutive "\n" with a single "\n"
    text = re.sub(r'\n+', '\n', text)
    date = root.find(".//NumericDate").text
    year, month = date.split("-")[:2]
    title = root.find(".//SortTitle").text
    loc = root.find(".//Qualifier").text
    
    sentences = text_to_sentences(text).split("\n")  # Assuming text_to_sentences just splits by newlines
    
    total_sents += len(sentences)
    inflation_sents = []
    for sent in sentences:
        if check_inflation(sent):
            # inflation_sents += 1
            inflation_sents.append(sent)
            
    file_id = file.split(".")[0]
    
    return none_text, total_sents, inflation_sents, f"{year}-{month}", file_id, title, loc

In [22]:
files = [file for file in os.listdir(base_data_dir) if file.endswith(".xml")]

with Pool(cpu_count()) as pool:
        results = list(tqdm(pool.imap(process_file, files), total=len(files)))
    
# Aggregate results
none_text = sum(result[0] for result in results)
total_sents = sum(result[1] for result in results)
# inflation_sents = [res for result in results for res in result[2]]
inflation_sents = [result[2] for result in results]
# n_inflation_sents = len(inflation_sents)
year_month = [result[3] for result in results]
file_id = [result[4] for result in results]

df = pd.DataFrame({"file_id": file_id, "year_month": year_month, "text": inflation_sents})
df= df.explode("text").reset_index(drop=True)
df = df.dropna()

file_path = f"{out_dir}/test_sents.csv"
df.to_csv(file_path, sep="\t", index=False)

print(f"Total sentences processed: {total_sents}")
print(f"Total sentences mentioning inflation: {len(df)}")
print(f"Total files with no hidden text: {none_text}")

  0%|          | 0/2 [00:00<?, ?it/s]

Atlanta, Ga.Atlanta, Ga.



100%|██████████| 2/2 [00:00<00:00, 248.25it/s]

Total sentences processed: 40
Total sentences mentioning inflation: 0
Total files with no hidden text: 0
